## Étude du corpus PARSEME

In [ ]:
# -*- coding: utf-8 -*-
# Jianying LIU

In [12]:
# télécharger les données
# À lancer une seule fois

!curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3367{/README.md,/trial.tgz,/bin.tgz,/FR.tgz,/ZH.tgz}
!tar zxvf FR.tgz
!tar zxvf ZH.tgz

x ZH/
x ZH/dev-stats.md
x ZH/dev.cupt
x ZH/README.md
x ZH/test-stats.md
x ZH/test.blind.cupt
x ZH/test.cupt
x ZH/train-stats.md
x ZH/train.cupt


In [3]:
import re
import json
from collections import Counter
import pandas as pd

liste_fic = ["train", "dev", "test"]

In [4]:
def pron_issame(dico_a,dico_b):
    for i in dico_a.keys():
        if i not in dico_b.keys():
            return False
        if dico_a[i] != dico_b[i]:
            return False
    return True

def calculate_same_ratio(dico):
    groups = [dico[0]]
    groups_nbre = [1]

    for i in range(len(dico)-1):
        if pron_issame(dico[i], dico[i+1]):
            indice_dicoi = groups.index(dico[i])
            groups_nbre[indice_dicoi] += 1
        if not pron_issame(dico[i], dico[i+1]):
            groups.append(dico[i+1])
            groups_nbre.append(1)
            
    nbre_max_same = max(groups_nbre)
    indice = groups_nbre.index(max(groups_nbre))
    
    return groups[indice],nbre_max_same

In [5]:
def restructurer_cupt(fichier_input):
    """
    Récupère le fichier cupt sous forme de liste de dictionnaires.
    Pour chaque phrase on a:
    - "sent_id" : l'identifiant de la phrase
    - "text" : le texte de la phrase
    - "MWE" : un dictionnaire avec en clé l'ID de la MWE dans la phrase et en
            valeur le type et la liste de tokens de la MWE.
    """
    with open(fichier_input, encoding="utf8") as filein:
        text = filein.read()
    text = re.sub("# global.columns = ID FORM LEMMA UPOS XPOS FEATS HEAD"
                  " DEPREL DEPS MISC PARSEME:MWE\n", "", text.strip())
    text = re.sub(r"# text_en =[^\n]+\n", "", text)
    text = re.sub(r"# newdoc id =[^\n]+\n", "", text)
    liste_sents = text.split("\n\n")

    liste_sent_sortie = []

    for sent in liste_sents:
        lignes = sent.split("\n")
        source_sent_id = re.match(r"# source_sent_id = (.+)", lignes[0])
        source_sent_id = source_sent_id.group(1)
        text_sent = re.match(r"# text = (.+)", lignes[1])
        text_sent = text_sent.group(1)

        dico_mwe = {}  # Dico interne de phrase
        for token_line in lignes[2:]:
            token_info_liste = token_line.split("\t")
            indice_token = token_info_liste[0]
            form_token = token_info_liste[1]
            lemme_token = token_info_liste[2]
            pos_token = token_info_liste[3]
            mwe_token = token_info_liste[10]

            # Traitement de colonne MWE
            col_sep = mwe_token.split(";")
            for mark_mwe in col_sep:
                if mark_mwe == "*":
                    continue
                if ":" in mark_mwe:
                    id_mwe = mark_mwe.split(":")[0]
                    type_mwe = mark_mwe.split(":")[1]
                    dico_mwe[id_mwe] = {"type": type_mwe,
                                        "tokens_list": [form_token],
                                        "lemmes_list": [lemme_token],
                                        "pos_list": [pos_token],
                                        "indice_list": [int(indice_token)],
                                        "tokens_nbre": 1}
                    if pos_token == "PRON":
                        dico_mwe[id_mwe]["pron_info"] = {lemme_token.lower():form_token.lower()}
                else:
                    dico_mwe[mark_mwe]["tokens_list"].append(form_token)
                    dico_mwe[mark_mwe]["lemmes_list"].append(lemme_token)
                    dico_mwe[mark_mwe]["pos_list"].append(pos_token)
                    dico_mwe[mark_mwe]["indice_list"].append(int(indice_token))
                    dico_mwe[mark_mwe]["tokens_nbre"] = dico_mwe[mark_mwe]["tokens_nbre"] + 1
                    if pos_token == "PRON":
                        if "pron_info" in dico_mwe[mark_mwe].keys():
                            dico_mwe[mark_mwe]["pron_info"][lemme_token.lower()]=form_token.lower()
                        else:
                            dico_mwe[id_mwe]["pron_info"] = {lemme_token.lower():form_token.lower()}
        if dico_mwe != {}:
            liste_sent_sortie.append({"sent_id": source_sent_id,
                                      "text": text_sent, "MWE": dico_mwe})

    return liste_sent_sortie


In [6]:
def get_one_type_mwe_list(liste_sents_sortie, type_mwe):
    """
    Args:
        liste_sents_sortie (liste de dict): chaque phrase des trois fichiers
        (sent_id, text et MWE)
        type_mwe (str): le type demandé

    Returns:
        - liste_one_type (liste de dict): dictionnaire de MWEs par type
          Pour chaque MWE on a:
            - 'content': la liste des tokens
            - 'type' : le type de la MWE
            - 'contexte_sent' : la phrase qui contient la MWE

        - len(all_mwe) (int): le nombre total de MWEs
    """
    all_mwe = []
    liste_one_type = []
    for sent in liste_sents_sortie:
        mwes = sent["MWE"]
        for dico in mwes.values():
            ecart = 0
            if dico["tokens_nbre"] > 1:
                for i in range(len(dico["indice_list"])-1):
                    ecart = max(ecart, dico["indice_list"][i+1] - dico["indice_list"][i] - 1)
            if "pron_info" in dico.keys():
                pron_etat = dico["pron_info"]   # {lemme: form}
            else:
                pron_etat = ""
            all_mwe.append({"tokens": dico["tokens_list"],
                            "lemmes": dico["lemmes_list"],
                            "indices": dico["indice_list"],
                            "nbre_tokens": dico["tokens_nbre"],
                            "ecart_max": ecart,
                            "lemmes_cnt": Counter(dico["lemmes_list"]),
                            "type": dico["type"],
                            "pron_etat": pron_etat,
                            "nbre_pron":len(pron_etat),
                            "contexte_sent": sent["text"]})

    if type_mwe == "all":
        return all_mwe, len(all_mwe)
    else:
        for dico in all_mwe:
            if dico["type"] == type_mwe:
                liste_one_type.append(dico)
        return liste_one_type

In [7]:
def unifier_mwe_identique_pron(liste_mwe):
    """
    unifier différentes formes d'une MWE
    """
    dico_mwe = {}
    for i in liste_mwe:
        expr = i["lemmes_cnt"]
        flag_trouve = False
        for mwe, mwe_content in dico_mwe.items():
            expr_exist = mwe_content['expr_cnt']
            if expr == expr_exist and i['type'] == mwe_content['type']:
                flag_trouve = True
                dico_mwe[mwe]['nbre_occurrence'] += 1
                dico_mwe[mwe]['contextes'].append((str(i['tokens']), i['contexte_sent']))
                if i["pron_etat"] != "":
                    dico_mwe[mwe]["pron_dicts"].append(i["pron_etat"])
                break
        if not flag_trouve and i["pron_etat"] != "":
            expr_cle = " ".join(i['lemmes'])
            dico_mwe[expr_cle] = {'expr_cnt': i['lemmes_cnt'], 'type': i['type'],
                                  'contextes': [(str(i['tokens']), i['contexte_sent'])],
                                  'pron_dicts':[i["pron_etat"]],
                                  'nbre_occurrence': 1}
    for cle,mwe in dico_mwe.items():
        most_pron, nbre_max_same = calculate_same_ratio(mwe['pron_dicts'])
        dico_mwe[cle]["most_pron"]=most_pron
        dico_mwe[cle]["same_ratio"]= nbre_max_same/dico_mwe[cle]["nbre_occurrence"]
    return dico_mwe

In [8]:
def unifier_mwe_identique(liste_mwe):
    """
    unifier différentes formes d'une MWE
    """
    dico_mwe = {}
    for i in liste_mwe:
        expr = i["lemmes_cnt"]
        flag_trouve = False
        for mwe, mwe_content in dico_mwe.items():
            expr_exist = mwe_content['expr_cnt']
            if expr == expr_exist and i['type'] == mwe_content['type']:
                flag_trouve = True
                dico_mwe[mwe]['nbre_occurrence'] += 1
                dico_mwe[mwe]['contextes'].append((str(i['tokens']), i['contexte_sent']))
                break
        if not flag_trouve:
            expr_cle = " ".join(i['lemmes'])
            dico_mwe[expr_cle] = {'expr_cnt': i['lemmes_cnt'], 'type': i['type'],
                                  'contextes': [(str(i['tokens']), i['contexte_sent'])],
                                  'nbre_occurrence': 1}
    return dico_mwe

In [9]:
def calculate_same_ratio_order(listes):
    listes_concat = []
    for x in listes:
        listes_concat.append(" ".join(x))
    groups = []
    groups_nbre = []

    for i in listes_concat:
        if i not in groups:
            groups.append(i)
            groups_nbre.append(1)
        else:
            indice_listi = groups.index(i)
            groups_nbre[indice_listi] += 1
            
    nbre_max_same = max(groups_nbre)
    indice = groups_nbre.index(max(groups_nbre))   
    return groups[indice],groups,nbre_max_same

In [10]:
def unifier_mwe_identique_order(liste_mwe):
    """
    unifier différentes formes d'une MWE
    """
    dico_mwe = {}
    for i in liste_mwe:
        expr = i["lemmes_cnt"]
        flag_trouve = False
        for mwe, mwe_content in dico_mwe.items():
            expr_exist = mwe_content['expr_cnt']
            if expr == expr_exist and i['type'] == mwe_content['type']:
                flag_trouve = True
                dico_mwe[mwe]['nbre_occurrence'] += 1
                dico_mwe[mwe]['contextes'].append((str(i['tokens']), i['contexte_sent']))
                dico_mwe[mwe]["order_lemme"].append(i["lemmes"])
                break
        if not flag_trouve:
            expr_cle = " ".join(i['lemmes'])
            dico_mwe[expr_cle] = {'expr_cnt': i['lemmes_cnt'], 'type': i['type'],
                                  'contextes': [(str(i['tokens']), i['contexte_sent'])],
                                  'order_lemme':[i["lemmes"]],
                                  'nbre_occurrence': 1}
    for cle,mwe in dico_mwe.items():
        most_order, orders, nbre_max_same = calculate_same_ratio_order(mwe['order_lemme'])
        dico_mwe[cle]["most_order"]=most_order
        dico_mwe[cle]["order_lst"]=orders
        dico_mwe[cle]["order_same_ratio"]= nbre_max_same/dico_mwe[cle]["nbre_occurrence"]
    return dico_mwe

### lancer ce bloc pour le chinois

In [13]:
# ZH
corpus = "zh"
liste = []
for fic_name in liste_fic:
    fic = f"ZH/{fic_name}.cupt"
    print("Charger fichier :",fic)
    liste_partie = restructurer_cupt(fic)
    liste.extend(liste_partie)
liste_all, len_all = get_one_type_mwe_list(liste, "all")
df_all = pd.DataFrame(liste_all)

Charger fichier : ZH/train.cupt
Charger fichier : ZH/dev.cupt
Charger fichier : ZH/test.cupt


### lancer ce bloc pour le français

In [11]:
# FR
corpus = "fr"
liste = []
for fic_name in liste_fic:
    fic = f"FR/{fic_name}.cupt"
    print("Charger fichier :",fic)
    liste_partie = restructurer_cupt(fic)
    liste.extend(liste_partie)
liste_all, len_all = get_one_type_mwe_list(liste, "all")
df_all = pd.DataFrame(liste_all)

Charger fichier : FR/train.cupt
Charger fichier : FR/dev.cupt
Charger fichier : FR/test.cupt


### nombres des MWE par type

In [16]:
print("Répartition des MWE :")
cat_count = df_all.type.value_counts()
print(cat_count)

Répartition des MWE :
VID          2156
LVC.full     1878
IRV          1501
LVC.cause      97
MVC            22
Name: type, dtype: int64


### Répartition des EPV

In [32]:
liste_all_divers = unifier_mwe_identique(liste_all)
df_divers = pd.DataFrame(liste_all_divers)
df_divers = df_divers.T
df_divers

,expr_cnt,type,contextes,nbre_occurrence
tenir réunion,"{'tenir': 1, 'réunion': 1}",LVC.full,"[(['Tenue', 'réunion'], Tenue chez Vanderdries...",6
il se agir,"{'il': 1, 'se': 1, 'agir': 1}",VID,"[(['il', ""s'"", 'agit'], En effet, s'il s'agit ...",140
il y avoir,"{'il': 1, 'y': 1, 'avoir': 1}",VID,"[(['il', 'y', 'a'], Le rez-de-chaussée est étr...",213
atteindre cancer,"{'atteindre': 1, 'cancer': 1}",LVC.full,"[(['atteinte', 'cancer'], L'une d'elles, une L...",4
avoir question,"{'avoir': 1, 'question': 1}",LVC.full,"[(['avez', 'questions'], - Si vous avez d'autr...",1
...,...,...,...,...
avoir un prix,"{'avoir': 1, 'un': 1, 'prix': 1}",VID,"[(['a', 'un', 'prix'], C'est plus cher que la ...",1
se pourvoir en cassation,"{'se': 1, 'pourvoir': 1, 'en': 1, 'cassation': 1}",VID,"[([""s'"", 'pourvu', 'en', 'cassation'], En févr...",1
signer prolongation,"{'signer': 1, 'prolongation': 1}",LVC.full,"[(['signe', 'prolongation'], En août 2010, il ...",1
faire retour,"{'faire': 1, 'retour': 1}",LVC.full,"[(['fait', 'retour'], En août 2010, il signe u...",1


In [34]:
if corpus == "zh":
    liste_type = ["LVC.full", "LVC.cause", "VID", "VPC.semi", "MVC"] #ZH
if corpus == "fr":
    liste_type = ["LVC.full", "LVC.cause", "VID", "IRV", "MVC"] #FR

for type_mwe in liste_type:
    print(f"### {type_mwe} ###")
    df_souscat = df_divers.loc[df_divers["type"] == type_mwe]
    sorted_df = df_souscat.sort_values(by='nbre_occurrence', ascending=False)
    print (sorted_df.head(10)["nbre_occurrence"])

### LVC.full ###
avoir droit         50
jouer rôle          45
avoir besoin        38
faire appel         32
avoir effet         19
faire apparition    17
signer contrat      17
poser question      17
match jouer         16
avoir chance        14
Name: nbre_occurrence, dtype: object
### LVC.cause ###
donner impression        4
donner occasion          4
entraîner baisse         3
entraîner mort           3
entraîner formation      2
apporter amélioration    2
donner opportunité       2
apporter solution        2
causer extinction        1
entraîner poursuite      1
Name: nbre_occurrence, dtype: object
### VID ###
il y avoir        213
il falloir        168
il se agir        140
avoir lieu        104
faire partie       98
compte tenir       37
faire le objet     35
il convenir        29
faire face         25
prendre part       25
Name: nbre_occurrence, dtype: object
### IRV ###
se trouver      104
se dérouler      66
se situer        52
se rendre        50
se produire      45
se élever 

### Analyse de la discontinuité

In [18]:
print("Répartition des discontinuités en général :")
ecart_count = df_all.ecart_max.value_counts()
print("Écart maximal entre token dans un MWE :")
print(ecart_count)
print("La proportion des MWE contigües :", ecart_count[0]/ecart_count.sum())

Répartition des discontinuités en général :
Écart maximal entre token dans un MWE :
0     3316
1     1535
2      462
3      156
4       76
5       40
6       20
8       11
9        9
7        8
10       6
11       3
15       2
12       2
16       2
13       2
42       1
14       1
20       1
19       1
Name: ecart_max, dtype: int64
La proportion des MWE contigües : 0.5864874425185709


In [19]:
df_compound = df_all.loc[df_all["ecart_max"] == 0]
distribution_cat = df_compound.type.value_counts()
print("continuous MWE distribution:")
print(distribution_cat)

continuous MWE distribution:
VID          1660
IRV          1283
LVC.full      347
MVC            21
LVC.cause       5
Name: type, dtype: int64


In [20]:
df_ecart1 = df_all.loc[df_all["ecart_max"] >= 1] #选取特定值
distribution_cat = df_ecart1.type.value_counts()
print("MWE ecart >=1 distribution:")
print(distribution_cat)

MWE ecart >=1 distribution:
LVC.full     1531
VID           496
IRV           218
LVC.cause      92
MVC             1
Name: type, dtype: int64


In [27]:
# Discontinuité pour chaque catégorie
print("Discontinuité pour chaque catégorie :")
if corpus == "zh":
    liste_type = ["LVC.full", "LVC.cause", "VID", "VPC.semi", "MVC"] #ZH
if corpus == "fr":
    liste_type = ["LVC.full", "LVC.cause", "VID", "IRV", "MVC"] #FR
for type_mwe in liste_type:
    df_souscat = df_all.loc[df_all["type"] == type_mwe]
    distribution_count = df_souscat.ecart_max.value_counts()
    print(f"### {type_mwe} ###")
    print(distribution_count)
    print("Proportion de MWE contigües dans cette catégorie : ", distribution_count[0]/distribution_count.sum())

Discontinuité pour chaque catégorie :
### LVC.full ###
1     917
0     347
2     341
3     116
4      65
5      35
6      15
8      11
9       7
7       7
10      5
11      3
13      2
15      2
16      2
12      1
19      1
20      1
Name: ecart_max, dtype: int64
Proportion de MWE contigües dans cette catégorie :  0.1847710330138445
### LVC.cause ###
1     51
2     18
3      9
0      5
4      5
6      3
5      2
9      2
10     1
12     1
Name: ecart_max, dtype: int64
Proportion de MWE contigües dans cette catégorie :  0.05154639175257732
### VID ###
0     1660
1      387
2       71
3       26
4        5
6        2
5        2
14       1
42       1
7        1
Name: ecart_max, dtype: int64
Proportion de MWE contigües dans cette catégorie :  0.7699443413729128
### IRV ###
0    1283
1     179
2      32
3       5
4       1
5       1
Name: ecart_max, dtype: int64
Proportion de MWE contigües dans cette catégorie :  0.854763491005996
### MVC ###
0    21
1     1
Name: ecart_max, dtype: int64
P

### Analyse du nombre de token dans chaque MWE

In [39]:
# Nombre de tokens
print("Répartition des nombres de tokens en général :")
nbre_tokens_count = df_all.nbre_tokens.value_counts()
print(nbre_tokens_count)
print(nbre_tokens_count.loc[1]/nbre_tokens_count.sum())
# liste_type = ["LVC.full", "LVC.cause", "VID", "VPC.semi", "MVC"] #ZH
liste_type = ["LVC.full", "LVC.cause", "VID", "IRV", "MVC"] #FR
for type_mwe in liste_type:
    df_souscat = df_all.loc[df_all["type"] == type_mwe]
    distribution_count = df_souscat.nbre_tokens.value_counts()
    print(f"### {type_mwe} ###")
    print(distribution_count)
    try:
        print(distribution_count.loc[1]/distribution_count.sum())
    except KeyError:
        print("No single token MWE")

Répartition des nombres de tokens en général :
2    4351
3    1048
4     181
5      34
6      28
7       6
1       5
8       1
Name: nbre_tokens, dtype: int64
0.0008843296781039972
### LVC.full ###
2    1757
3      74
4      43
6       2
5       2
Name: nbre_tokens, dtype: int64
No single token MWE
### LVC.cause ###
2    97
Name: nbre_tokens, dtype: int64
No single token MWE
### VID ###
2    977
3    973
4    136
5     32
6     26
7      6
1      5
8      1
Name: nbre_tokens, dtype: int64
0.002319109461966605
### IRV ###
2    1498
4       2
3       1
Name: nbre_tokens, dtype: int64
No single token MWE
### MVC ###
2    22
Name: nbre_tokens, dtype: int64
No single token MWE


In [40]:
df_all.loc[df_all["nbre_tokens"] == 1] # la seule 1 token MWE en français

,tokens,lemmes,indices,nbre_tokens,ecart_max,lemmes_cnt,type,contexte_sent
1399,[contre-indiqué],[contre-indiquer],[3],1,0,{'contre-indiquer': 1},VID,Aclasta est contre-indiqué chez la femme encei...
3404,[contre-indiqué],[contre-indiquer],[3],1,0,{'contre-indiquer': 1},VID,Angiox est contre-indiqué chez les patients :
3483,[contre-indiqué],[contre-indiquer],[3],1,0,{'contre-indiquer': 1},VID,Aclasta est contre-indiqué pendant la grossess...
4255,[contre-indiqué],[contre-indiquer],[3],1,0,{'contre-indiquer': 1},VID,Angiox est contre-indiqué chez les patients at...
4905,[contre-indiqué],[contre-indiquer],[3],1,0,{'contre-indiquer': 1},VID,Aclasta est contre-indiqué chez les patients a...


### Analyse des pronoms (seulement pour le français)

In [35]:
# dataframe des MWE contenant au moins un pronom
df_pron = df_all[df_all.pron_etat != ""]
df_pron

,tokens,lemmes,indices,nbre_tokens,ecart_max,lemmes_cnt,type,pron_etat,nbre_pron,contexte_sent
1,"[il, s', agit]","[il, se, agir]","[5, 6, 7]",3,0,"{'il': 1, 'se': 1, 'agir': 1}",VID,"{'il': 'il', 'se': 's''}",2,"En effet, s'il s'agit à l'origine de formes fé..."
2,"[il, y, a]","[il, y, avoir]","[7, 8, 9]",3,0,"{'il': 1, 'y': 1, 'avoir': 1}",VID,"{'il': 'il', 'y': 'y'}",2,"Le rez-de-chaussée est étroit, mais il y a une..."
12,"[se, compose]","[se, composer]","[2, 3]",2,0,"{'se': 1, 'composer': 1}",IRV,{'se': 'se'},1,Elle se compose de pratiques diverses soutenan...
13,"[il, conviendrait]","[il, convenir]","[7, 8]",2,0,"{'il': 1, 'convenir': 1}",VID,{'il': 'il'},1,Le rapport note néanmoins qu'« il conviendrait...
14,"[s', interroger]","[se, interroger]","[10, 11]",2,0,"{'se': 1, 'interroger': 1}",IRV,{'se': 's''},1,Le rapport note néanmoins qu'« il conviendrait...
...,...,...,...,...,...,...,...,...,...,...
5644,"[s', engagea]","[se, engager]","[2, 3]",2,0,"{'se': 1, 'engager': 1}",IRV,{'se': 's''},1,Il s'engagea dans l'armée prussienne et en 180...
5645,"[se, rendre]","[se, rendre]","[25, 26]",2,0,"{'se': 1, 'rendre': 1}",IRV,{'se': 'se'},1,"Le gîte ""A la Découverte"" est un havre de paix..."
5646,"[s', avère]","[se, avérer]","[3, 4]",2,0,"{'se': 1, 'avérer': 1}",IRV,{'se': 's''},1,"Leur écoute s'avère plutôt dispensable, même s..."
5649,"[se, rendaient]","[se, rendre]","[24, 25]",2,0,"{'se': 1, 'rendre': 1}",IRV,{'se': 'se'},1,Il les convainc de rentrer au Mali avec le max...


In [36]:
cat_count = df_pron.type.value_counts()
print("Nombre de MWE contenant pronom par type:")
print(cat_count)

Nombre de MWE contenant pronom par type:
IRV         1500
VID          781
LVC.full       3
Name: type, dtype: int64


In [37]:
df_pron.loc[:,"count"]= df_pron["pron_etat"].map(len)
# Montrer les MWE contenant 3 pronoms
df_pron[df_pron["count"]==3]

D:\Programs\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\Programs\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,tokens,lemmes,indices,nbre_tokens,ecart_max,lemmes_cnt,type,pron_etat,nbre_pron,contexte_sent,count
32,"[il, n', en, est, rien]","[il, ne, en, être, rien]","[19, 20, 21, 22, 23]",5,0,"{'il': 1, 'ne': 1, 'en': 1, 'être': 1, 'rien': 1}",VID,"{'il': 'il', 'en': 'en', 'rien': 'rien'}",3,"L'enquête, menée par une équipe de chercheurs ...",3
1961,"[Qu', en, est, -il]","[que, en, être, il]","[1, 2, 3, 4]",4,0,"{'que': 1, 'en': 1, 'être': 1, 'il': 1}",VID,"{'que': 'qu'', 'en': 'en', 'il': '-il'}",3,Qu'en est-il ?,3
2102,"[Qu', en, est, -il]","[que, en, être, il]","[1, 2, 3, 4]",4,0,"{'que': 1, 'en': 1, 'être': 1, 'il': 1}",VID,"{'que': 'qu'', 'en': 'en', 'il': '-il'}",3,Qu'en est-il de la préparation du budget allou...,3


In [39]:
## l'état des pronoms
# most_pron : les formes de pronoms les plus fréquentes dans cette expression
# same_ratio : la proportion de cette combinaison la plus fréquente parmi toutes les occurrences
liste_all_divers = unifier_mwe_identique_pron(liste_all)
df_divers = pd.DataFrame(liste_all_divers)
df_divers = df_divers.T
df_divers

,expr_cnt,type,contextes,pron_dicts,nbre_occurrence,most_pron,same_ratio
il se agir,"{'il': 1, 'se': 1, 'agir': 1}",VID,"[(['il', ""s'"", 'agit'], En effet, s'il s'agit ...","[{'il': 'il', 'se': 's''}, {'il': 'il', 'se': ...",140,"{'il': 'il', 'se': 's''}",0.957143
il y avoir,"{'il': 1, 'y': 1, 'avoir': 1}",VID,"[(['il', 'y', 'a'], Le rez-de-chaussée est étr...","[{'il': 'il', 'y': 'y'}, {'il': 'il', 'y': 'y'...",213,"{'il': 'il', 'y': 'y'}",0.934272
se composer,"{'se': 1, 'composer': 1}",IRV,"[(['se', 'compose'], Elle se compose de pratiq...","[{'se': 'se'}, {'se': 'se'}, {'se': 'se'}, {'s...",11,{'se': 'se'},1.0
il convenir,"{'il': 1, 'convenir': 1}",VID,"[(['il', 'conviendrait'], Le rapport note néan...","[{'il': 'il'}, {'il': 'il'}, {'il': 'il'}, {'i...",29,{'il': 'il'},1.0
se interroger,"{'se': 1, 'interroger': 1}",IRV,"[([""s'"", 'interroger'], Le rapport note néanmo...","[{'se': 's''}, {'se': 's''}, {'se': 's''}, {'s...",4,{'se': 's''},1.0
...,...,...,...,...,...,...,...
se auto-ériger,"{'se': 1, 'auto-ériger': 1}",IRV,"[([""s'"", 'auto-érige'], Par cette action, il s...",[{'se': 's''}],1,{'se': 's''},1.0
se désolidariser,"{'se': 1, 'désolidariser': 1}",IRV,"[([""s'"", 'désolidarisée'], - L'actuelle direct...",[{'se': 's''}],1,{'se': 's''},1.0
se risquer,"{'se': 1, 'risquer': 1}",IRV,"[(['se', 'risquer'], On peut tenter, avec la b...",[{'se': 'se'}],1,{'se': 'se'},1.0
se accroupir,"{'se': 1, 'accroupir': 1}",IRV,"[([""s'"", 'accroupissant'], Les danseurs effect...",[{'se': 's''}],1,{'se': 's''},1.0


In [40]:
print("Nombre de MWE contenant pronoms :", len(df_divers))
print("Nombre de MWE contenant pronoms (par type):")
df_divers.type.value_counts()

Nombre de MWE contenant pronoms : 388
Nombre de MWE contenant pronoms (par type):


IRV         282
VID         103
LVC.full      3
Name: type, dtype: int64

In [41]:
a = df_divers[df_divers.nbre_occurrence>1].same_ratio.value_counts()
print("Nombre de MWE pour chaque ratio d'identité des pronoms :")
print(a)
print("Proportion des MWE dont les pronoms sont toujours les mêmes formes :", 121/sum(a))

Nombre de MWE pour chaque ratio d'identité des pronoms :
1.000000    121
0.500000     16
0.666667     11
0.750000      7
0.714286      5
0.250000      4
0.800000      4
0.333333      3
0.400000      3
0.818182      3
0.833333      2
0.740741      1
0.481481      1
0.742424      1
0.764706      1
0.680000      1
0.827381      1
0.538462      1
0.600000      1
0.789474      1
0.688889      1
0.555556      1
0.957143      1
0.935484      1
0.909091      1
0.942308      1
0.934272      1
0.727273      1
0.785714      1
0.851852      1
0.904762      1
0.222222      1
0.777778      1
0.857143      1
0.700000      1
0.692308      1
0.619048      1
Name: same_ratio, dtype: int64
Proportion des MWE dont les pronoms sont toujours les mêmes formes : 0.5902439024390244


### Analyse des pronoms dans VID (expression distincte)

In [42]:
df_divers_plus1 = df_divers[df_divers.nbre_occurrence>1]
print("Nombre de MWE contenant pronoms et qui se répète au moins 1 fois :", len(df_divers_plus1))

print("Nombre de MWE contenant pronoms et qui se répète au moins 1 fois (par type):", len(df_divers_plus1))
print(df_divers_plus1.type.value_counts())

print("Pour le type VID :")
df_vid_pron = df_divers_plus1[df_divers_plus1.type == "VID"]
b = df_vid_pron.same_ratio.value_counts()
print("pronoms_same_raio décompte :")
print(b)
print("Nombre total de VID ayant pronom :", sum(b))
print("Proportion de MWE ayant pronoms inchangeables :", 30/45)

Nombre de MWE contenant pronoms et qui se répète au moins 1 fois : 205
Nombre de MWE contenant pronoms et qui se répète au moins 1 fois (par type): 205
IRV    160
VID     45
Name: type, dtype: int64
Pour le type VID :
pronoms_same_raio décompte :
1.000000    30
0.500000     5
0.750000     3
0.957143     1
0.934272     1
0.700000     1
0.714286     1
0.818182     1
0.400000     1
0.827381     1
Name: same_ratio, dtype: int64
Nombre total de VID ayant pronom : 45
Proportion de MWE ayant pronoms inchangeables : 0.6666666666666666


In [43]:
# pour voir tous les VID dont les pronoms ont changés
df_vid_pron[df_vid_pron.same_ratio != 1]["expr_cnt"]

il se agir                        {'il': 1, 'se': 1, 'agir': 1}
il y avoir                        {'il': 1, 'y': 1, 'avoir': 1}
il falloir                              {'il': 1, 'falloir': 1}
il être question            {'il': 1, 'être': 1, 'question': 1}
se faire remarquer        {'se': 1, 'faire': 1, 'remarquer': 1}
se faire il                      {'se': 1, 'faire': 1, 'il': 1}
se poser question          {'se': 1, 'poser': 1, 'question': 1}
il y avoir lieu        {'il': 1, 'y': 1, 'avoir': 1, 'lieu': 1}
se rendre compte            {'se': 1, 'rendre': 1, 'compte': 1}
tel être le cas        {'tel': 1, 'être': 1, 'le': 1, 'cas': 1}
il en être                        {'il': 1, 'en': 1, 'être': 1}
quel ne être pas      {'quel': 1, 'ne': 1, 'être': 1, 'pas': 1}
ça faire                                  {'ça': 1, 'faire': 1}
se en prendre                  {'se': 1, 'en': 1, 'prendre': 1}
que importer                          {'que': 1, 'importer': 1}
Name: expr_cnt, dtype: object

In [44]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',50)
pd.set_option('display.max_colwidth',15)

### Analyse de l'ordre de tokens dans un MWE

In [45]:
# l'état de l'ordre
liste_all_divers = unifier_mwe_identique_order(liste_all)
df_divers = pd.DataFrame(liste_all_divers)
df_divers = df_divers.T
df_divers

,expr_cnt,type,contextes,order_lemme,nbre_occurrence,most_order,order_lst,order_same_ratio
tenir réunion,{'tenir': 1...,LVC.full,"[(['Tenue',...","[[tenir, ré...",6,tenir réunion,[tenir réun...,0.5
il se agir,"{'il': 1, '...",VID,"[(['il', ""s...","[[il, se, a...",140,il se agir,[il se agir...,0.978571
il y avoir,"{'il': 1, '...",VID,"[(['il', 'y...","[[il, y, av...",213,il y avoir,[il y avoir...,0.967136
atteindre cancer,{'atteindre...,LVC.full,[(['atteint...,[[atteindre...,4,atteindre c...,[atteindre ...,1.0
avoir question,{'avoir': 1...,LVC.full,"[(['avez', ...","[[avoir, qu...",1,avoir question,[avoir ques...,1.0
...,...,...,...,...,...,...,...,...
avoir un prix,{'avoir': 1...,VID,"[(['a', 'un...","[[avoir, un...",1,avoir un prix,[avoir un p...,1.0
se pourvoir en cassation,"{'se': 1, '...",VID,"[([""s'"", 'p...","[[se, pourv...",1,se pourvoir...,[se pourvoi...,1.0
signer prolongation,{'signer': ...,LVC.full,"[(['signe',...","[[signer, p...",1,signer prol...,[signer pro...,1.0
faire retour,{'faire': 1...,LVC.full,"[(['fait', ...","[[faire, re...",1,faire retour,[faire retour],1.0


In [48]:
a = df_divers.order_same_ratio.value_counts()
print("Proportion des MWE avec un ordre inchangeable :", a[1]/sum(a))

Proportion des MWE avec un ordre inchangeable : 0.9367720465890182


In [49]:
# MWE apparaît plus 1 fois
df_occ_plus1 = df_divers[df_divers.nbre_occurrence>1]
df_occ_plus1

,expr_cnt,type,contextes,order_lemme,nbre_occurrence,most_order,order_lst,order_same_ratio
tenir réunion,{'tenir': 1...,LVC.full,"[(['Tenue',...","[[tenir, ré...",6,tenir réunion,[tenir réun...,0.5
il se agir,"{'il': 1, '...",VID,"[(['il', ""s...","[[il, se, a...",140,il se agir,[il se agir...,0.978571
il y avoir,"{'il': 1, '...",VID,"[(['il', 'y...","[[il, y, av...",213,il y avoir,[il y avoir...,0.967136
atteindre cancer,{'atteindre...,LVC.full,[(['atteint...,[[atteindre...,4,atteindre c...,[atteindre ...,1.0
avoir doute,{'avoir': 1...,LVC.full,"[(['avez', ...","[[avoir, do...",2,avoir doute,[avoir doute],1.0
...,...,...,...,...,...,...,...,...
avoir hypocalcémie,{'avoir': 1...,LVC.full,"[(['ayant',...","[[avoir, hy...",2,avoir hypoc...,[avoir hypo...,1.0
revêtir caractère,{'revêtir':...,LVC.full,[(['revêtai...,"[[revêtir, ...",2,revêtir car...,[revêtir ca...,1.0
fournir réponse,{'fournir':...,LVC.full,[(['fournir...,"[[fournir, ...",2,fournir rép...,[fournir ré...,0.5
avoir ennui,{'avoir': 1...,LVC.full,"[(['AUREZ',...","[[avoir, en...",2,avoir ennui,[avoir ennui],1.0


In [50]:
print("Nombre de MWE pour chaque ratio d'identité (parmi les MWE répétés :)")
b = df_occ_plus1.order_same_ratio.value_counts()
print(b)
print("Nombre de MWE qui répète :", len(df_occ_plus1))
print("Proportion des MWE dont l'ordre est inchangeable :",b[1]/len(df_occ_plus1))

Nombre de MWE pour chaque ratio d'identité (parmi les MWE répétés :)
1.000000    582
0.500000     46
0.666667     29
0.600000      4
0.750000      4
0.571429      3
0.833333      3
0.909091      2
0.857143      2
0.800000      2
0.928571      2
0.764706      1
0.923077      1
0.588235      1
0.727273      1
0.844444      1
0.978571      1
0.538462      1
0.941176      1
0.818182      1
0.875000      1
0.967136      1
0.785714      1
0.916667      1
0.952381      1
0.910714      1
0.648649      1
0.894737      1
Name: order_same_ratio, dtype: int64
Nombre de MWE qui répète : 696
Proportion des MWE dont l'ordre est inchangeable : 0.8362068965517241


In [51]:
# la répartition des MWE dans chaque type parmi les MWE répétés
df_occ_plus1.type.value_counts()

LVC.full     314
VID          210
IRV          160
LVC.cause      8
MVC            4
Name: type, dtype: int64

In [52]:
for cat in ["LVC.full","VID","IRV","LVC.cause","MVC"]:
    print("###########",cat,"###########")
    df_cat = df_occ_plus1[df_occ_plus1.type==cat]
    print("nombre EPV:", len(df_cat))
    ratios = df_cat.order_same_ratio.value_counts()
    print("Nombre de MWE dont l'ordre de tokens est inchangeable :", ratios[1])
    print("Proportion de ces MWE dans cette catégorie :", ratios[1] / len(df_cat))

########### LVC.full ###########
nombre EPV: 314
Nombre de MWE dont l'ordre de tokens est inchangeable : 217
Proportion de ces MWE dans cette catégorie : 0.6910828025477707
########### VID ###########
nombre EPV: 210
Nombre de MWE dont l'ordre de tokens est inchangeable : 196
Proportion de ces MWE dans cette catégorie : 0.9333333333333333
########### IRV ###########
nombre EPV: 160
Nombre de MWE dont l'ordre de tokens est inchangeable : 157
Proportion de ces MWE dans cette catégorie : 0.98125
########### LVC.cause ###########
nombre EPV: 8
Nombre de MWE dont l'ordre de tokens est inchangeable : 8
Proportion de ces MWE dans cette catégorie : 1.0
########### MVC ###########
nombre EPV: 4
Nombre de MWE dont l'ordre de tokens est inchangeable : 4
Proportion de ces MWE dans cette catégorie : 1.0
